# O projeto é uma análise de dados que deve conter as seguintes etapas:

•	Carregar e ler arquivos .csv, xlsx ou de um banco de dados;

•	Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo);

•	Realizar a codificação das variáveis categóricas conforme os valores das variáveis (label encoder, one hot encoder ou target encoder);

•	Normalizar as variáveis;

•	Avaliar a necessidade de realizar o balanceamento da variável alvo;

•	Tratar variáveis com alta correlação;

•	Realizar a seleção de variáveis;

•	Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;

•	Aplicar uma medida de avaliação do modelo.

•	Gerar uma visualização para o resultado conforme os tipos abaixo:
    - Agrupamento: gerar a visualização Scatter;
    - Classificação: gerar a matriz de confusão; 
    - Regressão: gerar a visualização da linha de saída.


In [14]:
import pandas as pd # O 'as' é um alias, um apelido. Ao invés de escrever 'pandas', escrevemos 'pd'
import numpy as np

# 1. Carregar o arquivo .csv usando o pd.read_csv()

In [15]:
df = pd.read_csv('Pokemon.csv') # Lê o arquivo .csv e armazena em um DataFrame
df.head() #  Traz os 5 primeiros registros do DataFrame

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# 2. Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo)

Não vimos necessidade de escalar as variáveis, pois o dataset é pequeno e autoexplicativo.

In [16]:
df.drop(['#'], axis=1, inplace=True) # Remove a coluna '#'; o axis 1 indica que é uma coluna, se fosse 0, indicaria que é uma linha; inplace=True indica que a alteração é feita no próprio DataFrame
colunas = ['nome', 'tipo_1', 'tipo_2', 'total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade', 'geracao', 'lendario'] # traduzindo as colunas e renomeando de uma forma mais amigável para o modelo, tirando espaços e maiúsculas
df.columns = colunas # Renomeia as colunas do DataFrame
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# 3. Realizar a codificação das variáveis categóricas de acordo com os valores das variáveis (label encoder, one hot encoder ou target encoder)

In [17]:
# Label Encoder

from sklearn.preprocessing import LabelEncoder # Importa a classe LabelEncoder do módulo preprocessing da biblioteca sklearn

le = LabelEncoder() # O LabelEncoder faz sentido quando a variável categórica é ordinal, ou seja, tem uma ordem natural. O pokemon tem mais 'peso' por ser lendário.

df['lendario'] = le.fit_transform(df['lendario']) # Transforma os valores da coluna lendario em valores numéricos [0-1]
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,0


# Normalizar as variáveis

In [18]:
# Antes, uma análise descritiva do DataFrame, é bom para observar valores discrepantes, outliers, e ter uma noção da distribuição dos dados.
df.describe()

,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
count,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000,800.00000
mean,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375,0.08125
std,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129,0.27339
min,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000,0.00000
25%,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000,0.00000
50%,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000,0.00000
75%,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000,0.00000
max,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000,1.00000


In [19]:
# Antes, vamos tratar os NaN da coluna do tipo_2, pois muitos pokemons não tem 2 tipos de categorias.

df['tipo_2'].fillna('None', inplace=True) # Preenche os valores NaN da coluna tipo_2 com 'None'
df.isna().sum() # Verifica se há valores NaN no DataFrame

C:\Users\Familia\AppData\Local\Temp\ipykernel_14940\1992376372.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tipo_2'].fillna('None', inplace=True) # Preenche os valores NaN da coluna tipo_2 com 'None'


nome          0
tipo_1        0
tipo_2        0
total         0
hp            0
ataque        0
defesa        0
sp_atq        0
sp_def        0
velocidade    0
geracao       0
lendario      0
dtype: int64

In [20]:
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,0


In [8]:
from sklearn.preprocessing import StandardScaler # Importa a classe StandardScaler do módulo preprocessing da biblioteca sklearn

# O standardScaler é uma técnica de normalização que transforma os dados para que a média seja 0 e o desvio padrão seja 1.
# Se o valor da variável for menor que a média, o valor transformado será negativo. Se for maior, será positivo. Quanto maior o número, mais próximo, ou até maior que 1 ele será.

scaler = StandardScaler() # Instancia o StandardScaler
# Vamos usar o Scaler, pois ele é mais sensível a outliers, o que é comum em pokemons lendários. 

# Para manter o DataFrame original, vamos criar um DataFrame com os valores normalizados
df_norm = df.copy() # Cria uma cópia do DataFrame
df_norm[['total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade']] = scaler.fit_transform(df_norm[['total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade']]) # Normaliza as colunas int64 e float64

df_norm.head()


,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,-0.976765,-0.950626,-0.924906,-0.797154,-0.239130,-0.248189,-0.801503,1,0
1,Ivysaur,Grass,Poison,-0.251088,-0.362822,-0.524130,-0.347917,0.219560,0.291156,-0.285015,1,0
2,Venusaur,Grass,Poison,0.749845,0.420917,0.092448,0.293849,0.831146,1.010283,0.403635,1,0
3,VenusaurMega Venusaur,Grass,Poison,1.583957,0.420917,0.647369,1.577381,1.503891,1.729409,0.403635,1,0
4,Charmander,Fire,None,-1.051836,-1.185748,-0.832419,-0.989683,-0.392027,-0.787533,-0.112853,1,0


In [9]:
df.head() # Como visto, manteve o df original.

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,0


In [10]:
# Contagem de Pokémon lendários
num_lendario = df["lendario"].sum()
num_lendario

65

In [11]:
# Contagem total de Pokémon
total_pokemon = len(df)
total_pokemon


800

In [12]:
# Proporção de Pokémon lendários. Em relação ao 
# total de pokemons podemos considerar que a classe lendario está desbalanceada.
proporcao_lendarios = num_lendario / total_pokemon

print(f"Proporção de Pokémon lendários: {proporcao_lendarios:.2%}")

Proporção de Pokémon lendários: 8.12%
